<a href="https://colab.research.google.com/github/vasudev49/bnn/blob/main/ImageCNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install ipdb
!pip install pyro-ppl

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/BNN/')


# !unzip 'test-20210504T120542Z-001.zip'
# !unzip 'Containers-20210504T120540Z-001.zip'

!ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Al-Sibahi_MNIST_1.ipynb	       HPOResults.gsheet
 BNN-FromKaggle.ipynb		       ImageCNN_MNIST.ipynb
 BNN_Linear_Regression_Example.ipynb   ImageCNN_RUST.ipynb
 cifar-10-batches-py-data	       ListOfHPOParams.csv
 Containers			       ListOfHPOParams.gsheet
 Containers-20210504T120540Z-001.zip   MNIST
'Copy of bnn_parasChopra.ipynb'        mnist-data
'Copy of HPOResults.csv'	       Pyro_intro.ipynb
'Copy of HPOResults.gsheet'	       Pyro_Poutine.ipynb
 data				       test
 HPOResults.csv			       test-20210504T120542Z-001.zip


In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
import functools
import operator
import ipdb
import pyro
import time
import random
from datetime import datetime, timezone
import pytz

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
pyro.set_rng_seed(0)

In [ ]:
# config:

img_height = 28
img_width = 28
input_dim = [1,img_height, img_width]

device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
n_epochs = 1
device = torch.device('cpu')
batch_size = 500
n_train_data=20000

In [ ]:
# def load_transform_split(trainDataDir, testDataDir, img_height = 180, img_width = 180, valid_size=.2, batch_size  = 5):
#   # ipdb.set_trace()
#   train_transform = transforms.Compose([transforms.ToTensor(),])
#   test_transform = transforms.Compose([transforms.ToTensor(),])
#   trainData= datasets.ImageFolder(trainDataDir,transform = train_transform)
#   testData = datasets.ImageFolder(testDataDir, transform = test_transform)  
#   trainLoader = torch.utils.data.DataLoader(trainData, batch_size= batch_size , shuffle=True)
#   testLoader = torch.utils.data.DataLoader(testData, batch_size= batch_size , shuffle=True)
#   return trainLoader, testLoader, trainData.class_to_idx, testData.class_to_idx
# trainLoader, testLoader, train_class_to_idx, test_class_to_idx =  load_transform_split("Containers", "test")
# # print(next(iter(trainLoader)))
trainData_ = datasets.MNIST('mnist-data', train=True, download=True,transform = transforms.Compose([transforms.ToTensor(),]))
len_trainData_ = trainData_.data.shape[0]
trainData, valData = torch.utils.data.random_split(trainData_, [n_train_data,len_trainData_-n_train_data])

testData = datasets.MNIST('mnist-data', train=False, download=True,transform = transforms.Compose([transforms.ToTensor(),] ))

train_loader = torch.utils.data.DataLoader(trainData, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(testData, batch_size=batch_size)

 
print(trainData.dataset.class_to_idx)
print(testData.class_to_idx)

len(trainData)
# trainDataSubset = torch.utils.data.RandomSampler(data_source=trainData, replacement=True, num_samples=20000, generator=None)

# for image, labels in trainDataSubset:
#   print(image.shape)
#   print(label.shape)
#  np.random.randint(0,10,5)

# np.random.seed(0)
# np.random.choice(range(10),5, replace=False)  

{'0 - zero': 0, '1 - one': 1, '2 - two': 2, '3 - three': 3, '4 - four': 4, '5 - five': 5, '6 - six': 6, '7 - seven': 7, '8 - eight': 8, '9 - nine': 9}
{'0 - zero': 0, '1 - one': 1, '2 - two': 2, '3 - three': 3, '4 - four': 4, '5 - five': 5, '6 - six': 6, '7 - seven': 7, '8 - eight': 8, '9 - nine': 9}


20000

In [ ]:
from torch.nn import NLLLoss 
class PlainCNN(nn.Module):
  def __init__(self, in_channels=1, out_channels=16, outdim=10, input_dim=input_dim):
    super(PlainCNN, self).__init__()
    # self.cuda()
    self.CNN_layer = nn.Sequential(
        nn.Conv2d(in_channels = in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=0),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True), 
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=out_channels, out_channels=4, kernel_size=3, stride=1, padding=0),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten()
        )
    num_features_from_CNN_layer = functools.reduce(operator.mul, list(self.CNN_layer(torch.rand(1, *input_dim)).shape))
    self.Linear_layer = nn.Sequential(
            nn.Linear(num_features_from_CNN_layer, outdim),
            # nn.LogSoftmax()
            )
  def forward(self, x):
      x = self.CNN_layer(x)
      x = self.Linear_layer(x)  ## note: outputs logits... not softmax.
      return x




In [ ]:

# model = PlainCNN()


# optimizer = optim.Adam(model.parameters(), lr=0.01)
# # lossfn=NLLLoss()
# lossfn = nn.CrossEntropyLoss() ## CrossEntropyLoss expects logits as inputs

# model = model.to(device)
# lossfn = lossfn.to(device)

# for i in range(n_epochs):
#   running_loss = 0
#   batch_no = 0
#   for images, labels in train_loader:
#     batch_no+=1
#     images = images.to(device)
#     labels = labels.to(device)
#     optimizer.zero_grad()
#     # ipdb.set_trace()
#     outputs= model(images)
#     loss = lossfn(outputs.squeeze(), labels)
#     loss.backward()
#     optimizer.step()
#     running_loss += loss.item()
#     # break;
#     if batch_no%50==0:
#         print (f'epoch: {i} Batch: {batch_no} loss: {running_loss}')

#how many params are there?
# sum(p.numel() for p in model.parameters())

# for images, labels in trainLoader:
#   print(model(images))

In [ ]:
# resultList =[]
# model_perf=0
# for image, labels in test_loader:
#   # ipdb.set_trace()
#   result = np.argmax(model(image.to(device)).detach().numpy(), axis=1)
#   resultList.extend(result.tolist())
#   model_perf += sum(labels.numpy() == result)

# resultList
# model_perf/len(test_loader.dataset)


# i=0
# bs = test_loader.batch_size
# for image, labels in test_loader:
#   for img in image:
#     # plt.imshow(img.permute(1,2,0).squeeze())
#     # plt.show()
#     print(f'Actual Label:{labels[i%bs]}, Predicted:{np.argmax(np.exp(resultList[i]))}')
#     i+=1
#   if i>10:
#     break
#     print("------------")

# len(test_loader)


## Bayesi-*fy* the CNN 

In [ ]:
from pyro.nn.module import to_pyro_module_
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch.nn as nn
# from pyro.infer.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import autoguide
from pyro.infer import SVI, Trace_ELBO, Predictive
from tqdm.auto import trange, tqdm
from pyro.optim import Adam as pyro_ka_Adam


# def __init__(self, in_channels=3, out_channels=16, input_dim = [3,img_height, img_width]):

class BayesianCNN(PyroModule):
  def __init__(self, CNN_with_priors):
    super().__init__()
    self.neural_network = CNN_with_priors
  def forward(self, X_train, y_train=None):
    # sigma = pyro.sample("sigma", dist.Uniform(0.,10.))
    mean = self.neural_network(X_train).squeeze()
    with pyro.plate("data", X_train.shape[0], device = device):
      # obs = pyro.sample("obs", dist.Categorical(logits = mean), obs=y_train)
      # obs =  pyro.sample("obs", dist.Categorical(logits = mean), obs=y_train)
      pyro.sample("obs", dist.Categorical(logits = mean), obs=y_train)

    return (mean)  
    # we will return this mean and while predicting these will be simulated too using the _RETURN return_sites in Predictive()
    # this mean is the logits of the classifier prediction of the 10 numbers

# dist.Categorical(logits = torch.tensor([1.,2.,3.]))
# pyro.sample("obs", dist.Categorical(logits = torch.tensor([1.,2.,3.])))

In [ ]:
# guide = autoguide.AutoDiagonalNormal(bcnn, init_scale=inital_variance)


In [ ]:
# bnn config

inital_variance = 0.01
n_epochs = 14
learning_rate = 0.001
n_samples=200
early_stopping_tol=0.005

In [ ]:
# from importlib import reload  
# import torch.optim as optim
# reload( torch.optim)
from torch.distributions import constraints


PlainCNNModel = PlainCNN()
PlainCNNModel = PlainCNNModel.to(device)
to_pyro_module_(PlainCNNModel)
for m in PlainCNNModel.modules():
    # print(f'--------------- module:{m} -----------------')
    for name, value in list(m.named_parameters(recurse=False)):
      # print(f'************* name: {name} *************')    
      # print(f'>>>>>> weight.shape: {value.shape}<<<<<<<<<')  
      setattr(m, name, PyroSample(prior=dist.Normal(0, 1)
                                        .expand(value.shape)
                                        .to_event(value.dim())
                                        ))
# PlainCNNModel = PlainCNNModel.to(device)
bcnn = BayesianCNN(PlainCNNModel)

guide = autoguide.AutoDiagonalNormal(bcnn, init_scale=inital_variance)


# guide = autoguide.AutoMultivariateNormal(bcnn, init_scale=0.01) # this gives an error: parameter loc has invalid values....
# guide = autoguide.AutoLowRankMultivariateNormal(bcnn)
bcnn = bcnn.to(device)
guide = guide.to(device)

# l = guide(torch.rand(1, *input_dim),1)
# print(f'Sum of Guide Variances BEFORE: {pyro.param("AutoDiagonalNormal.scale").sum()}')
# print(f'Sum of Guide Mean BEFORE: {pyro.param("AutoDiagonalNormal.loc").abs().sum()}')


# bcnn(images)
# ipdb.set_trace()
# svi = SVI(model = bcnn, guide = guide, optim=pyro_ka_Adam({'lr':0.001}), loss=Trace_ELBO())
# Reminder: dont mix positional and named argument to pyro functions....weird things happen.

from pyro.optim import ExponentialLR 
momentum = 0.1
num_steps = 10000
gamma = 0.1
lrd = gamma**(1/num_steps)

schedulerELR = ExponentialLR({'optimizer': torch.optim.Adam, 'optim_args': {'lr': learning_rate}, 'gamma': 0.1})
svi = SVI(model = bcnn, guide = guide,
          # optim=schedulerELR,
          # optim=pyro.optim.SGD({"lr": learning_rate, "momentum":momentum}),
          #  optim = pyro.optim.SGD({"lr":learning_rate}),
           optim = pyro.optim.Adam({"lr":learning_rate, 'betas' :(0.95, 0.999)}),
          #  optim = pyro.optim.ClippedAdam({"lr":learning_rate, 'lrd':lrd, 'betas' :(0.95, 0.999)}),
          #  optim = pyro.optim.ClippedAdam({"lr":learning_rate, 'betas' :(0.95, 0.999)}),
          loss=Trace_ELBO()
          # loss=Trace_ELBO(num_particles=10,vectorize_particles=True)
          # loss=Trace_ELBO(num_particles=10,vectorize_particles=False)

          )


# for n in pyro.get_param_store():#guide.named_parameters():
#   print(n)


def early_stopping(loss_list, tol=0.01, lookback=5):
  if(len(loss_list)<lookback+1):
    return (False)
  per_change =  np.array([np.abs(x/y-1) for (x,y) in zip(loss_list[-lookback:], loss_list[-(lookback+1):-1])])
  return(np.all(per_change<tol))


pyro.clear_param_store()

# latent_dim = 1790 # guide.latent_dim
# pyro.param("auto_loc", torch.randn(latent_dim))
# pyro.param("auto_scale", torch.ones(latent_dim),
#       constraint=constraints.positive)
# l = guide(image, labels)

# l = guide(torch.rand(1, *input_dim),1)
# print(f'Sum of Guide Variances BEFORE: {pyro.param("AutoDiagonalNormal.scale").sum()}')
# print(f'Sum of Guide Mean BEFORE: {pyro.param("AutoDiagonalNormal.loc").abs().sum()}')

start_training_time = time.time()
ELBO_loss = []

for i in range(n_epochs):
  running_loss = 0
  batch = 0
  epoch_loss=0
  for images, labels in train_loader:
    batch+=1
    # print(f'epoch: {i}  Batch: {batch}')
    images = images.to(device)
    labels = labels.float().to(device)
    # ipdb.set_trace() 
    elbo = svi.step(images, labels.squeeze())
    epoch_loss += elbo
    # logging.info("Elbo loss: {}".format(elbo))
    # if batch%50==0:
    #       print(f'epoch: {i} batch:{batch}  ELBO LOSS: {elbo} ')
    # print(f'------------------------------------- {i}')
    # break
  ELBO_loss.append(epoch_loss)
  lookback=5
  loss_list=ELBO_loss.copy()
  if(len(loss_list)>lookback+1):
    per_change =  np.array([np.abs(x/y-1) for (x,y) in zip(loss_list[-lookback:], loss_list[-(lookback+1):-1])])
    print(per_change)
  print(f'epoch: {i},  loss:{epoch_loss},  Guide VAriance: {pyro.param("AutoDiagonalNormal.scale").sum()}')
  if(early_stopping(ELBO_loss, early_stopping_tol)):
    print(f'EARLY stopping Triggered at {i}')
    break
 

# next(PlainCNNModel.parameters()).device
# for m in PlainCNNModel.modules():
#   for name, value in list(m.named_parameters(recurse=False)):
#     print(name)


# for n in pyro.get_param_store():#guide.named_parameters():
#   print(n)
plt.plot(ELBO_loss)

print(f'Sum of Guide Variances AFTER: {pyro.param("AutoDiagonalNormal.scale").sum()}')
print(f'Sum of Guide Mean AFTER: {pyro.param("AutoDiagonalNormal.loc").abs().sum()}')

# gg = pyro.param("AutoDiagonalNormal.scale").shape.numel()* inital_variance

end_training_time = time.time()
print(f'Training Time Taken: {(end_training_time - start_training_time)/60 }')
print(datetime.now(tz = pytz.timezone( 'Asia/Kolkata' )))

list(guide.parameters())

AttributeError: ignored

In [ ]:
# guide = autoguide.AutoDiagonalNormal(bcnn, init_scale=inital_variance)
# l = guide(torch.rand(1, *input_dim),1)

# pyro.params.param_store.ParamStoreDict.named_parameters(pyro.param)

# for k in pyro.get_param_store():
#   # print(k)
#   print(pyro.params.user_param_name(k))

# get_all_param_names()


In [ ]:
# TODO: how to use the stuff coming out of predictive.
# see what paras chopra did:
# https://github.com/paraschopra/bayesian-neural-network-mnist/blob/master/bnn.ipynb
# https://towardsdatascience.com/making-your-neural-network-say-i-dont-know-bayesian-nns-using-pyro-and-pytorch-b1c24e6ab8cd
# https://colab.research.google.com/drive/1pdYb15kdJLz8GkWNbc0mhb4GuEvDhgsn#scrollTo=5N0Ama80QkJ2
from datetime import datetime, timezone
import pytz
print(datetime.now(tz = pytz.timezone( 'Asia/Kolkata' )))

# Predict:
from pyro.infer import Predictive

start_prediction_time = time.time()

preds= Predictive(model = bcnn, guide = guide, num_samples = n_samples, return_sites=("_RETURN", "obs"))
correct_preds = 0

num_batches_to_run_for = 10
batch_num = 0
predictions_all = []
for img, labels in test_loader:
  samples = preds(img)
  predicted = np.argmax(samples['_RETURN'].mean(axis=[0]), axis=1)
  predictions_all.extend(predicted.tolist())
  correct_preds += (predicted == labels).sum()
  # if(num_batches_to_run_for == batch_num):
  #   break;
  batch_num +=1

# test_accuracy = correct_preds/ ((num_batches_to_run_for+1)*test_loader.batch_size) #len(test_loader.dataset)
test_accuracy = correct_preds/len(predictions_all)
print("---------------------------------")
print(f'test accuracy:{test_accuracy}')
end_prediction_time = time.time()
print(f'Prediction Time Taken: {start_prediction_time - end_prediction_time}')

samples.keys()
print(labels)
# print(samples['obs'])
# print(samples['_RETURN'])
# print(samples['_RETURN'].shape)



for i in range(5):
  # plt.imshow(img[i].squeeze())
  # plt.show()
  print(f'Actual label:{labels[i]}   Predictions: {samples["obs"][:,i]}')

# ADAM
# batSize SampleSize accuracy, epochs, variance
# 8, 200, 0.125 , 10epochs, 1
# 8, 1000, 0.10 , 10epochs, 1
# 32, 200, 0.15 , 10epochs, 1
# 32, 1000 0.16 , 10epochs, 1
# 64, 200  .13 , 10epochs, 1
# 64, 1000 .13 , 10epochs, 1
# 128, 200 0.86 , 10epochs, 1
#128, 200, 87, 40 epochs, 0.0001
#128, 200, 84, 80 epochs, 0.001
#128, 200, 83, 50 epochs, 0.01
#128, 200, 80, 30 epochs, 0.1

# 128, 1000 0.8656 , 10epochs, 1
# 1048 1000 78 , 10epochs, 1
# 1048 200 71 , 10epochs, 1
# 1048 200 83  20 epochs, 1
# 1048 200 90  40 epochs, 1
# 1048 200 90  80 epochs, 1
# 1048 1000 90  80 epochs, AutoguideVar=0.1
# 1048 2000 71, 10epochs, AutoguideVar=0.1
# 512 200 92, 40epochs, AutoguideVar=0.1
#10000, 200, 34, 40 epochs, AutoguideVar=0.1
#10000, 200, 78, 80 epochs, AutoguideVar=0.1

#5000, 200, 75, 40 epochs, AutoguideVar=0.1
#5000, 200, 76, 80 epochs, AutoguideVar=0.1
#5000, 200, 92, 80 epochs, AutoguideVar=0.05
#5000, 200, 95, 80 epochs, AutoguideVar=0.01
#5000, 200, 94, 80 epochs, AutoguideVar=0.001
#5000, 200, 96.6, 100 epochs, AutoguideVar=0.001
#5000, 200, 93, 100 epochs, AutoguideVar=0.1

#5000, 200, 97, 100 epochs, AutoguideVar=0.001
#5000, 100, 97, 100 epochs, AutoguideVar=0.001
#5000, 100, 97.5, 120 epochs, AutoguideVar=0.001
#5000, 100, 97.7, 100 epochs, AutoguideVar=0.0001

#50K, 200, 33, 55 epochs, AutoguideVar=0.001
#50K, 200, 52, 80 epochs, AutoguideVar=0.001
#50K, 200, 59, 120 epochs, AutoguideVar=0.001
#50K, 200, 58, 160 epochs, AutoguideVar=0.001
#50K, 1000, 59, 160 epochs, AutoguideVar=0.001

# running a jagged elo curve wala with particle thing to se if its smoother. Observation - the particle one was also jagged only.

In [ ]:
# samples = preds(img)
# predicted = np.argmax(samples['_RETURN'].mean(axis=[0]), axis=1)
sm = nn.LogSoftmax(dim=-1)
val = sm(samples['_RETURN'])
val.shape

np.exp(val[0,0,:]).sum()

In [ ]:
from matplotlib import colors


classes = ('0', '1', '2', '3','4', '5', '6', '7', '8', '9')


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    #plt.imshow(npimg,  cmap='gray')
    #fig.show(figsize=(1,1))    
    fig, ax = plt.subplots(figsize=(1, 1))
    # ax.imshow(npimg,  cmap='gray', interpolation='nearest')
    plt.imshow(npimg,  cmap='gray')
    plt.show()

def test_batch(preds, images, labels, plot=True):
    lsm = nn.LogSoftmax(dim=-1)
    sm = nn.Softmax(dim=-1)
    y = (lsm(preds(images)['_RETURN']))
    # y = lsm(lsm(preds(images)['_RETURN'])) ## double log soft max to coincide with what paras chopra has done. Doesnt seem to work.

    # y = (preds(images)['_RETURN']) #logits
    # prob_y = lsm(preds(images)['_RETURN']) # log_probabilities

    # ipdb.set_trace()
    predicted_for_images = 0
    correct_predictions=0

    for i in range(len(labels)):
    
        if(plot):
            print("-----------------------------------------------------------------------------------------------------")
            print("Real: ",labels[i].item())
            # fig, axs = plt.subplots(1, 10, sharex=True, sharey=True, figsize=(20,2))
            fig, axs = plt.subplots(1, 10, sharey=True, figsize=(20,2))
            
    
        all_digits_prob = []
    
        highted_something = False
    
        for j in range(len(classes)):
        
            highlight=False
        
            histo = []
            histo_exp = []
        
            for z in range(y.shape[0]):
                histo.append(y[z][i][j])
                histo_exp.append(np.exp(y[z][i][j]))
                # histo_exp.append(np.exp(prob_y[z][i][j]))
                
            
            prob = np.percentile(histo_exp, 50) #sampling median probability
        
            if(prob>0.2): #select if network thinks this sample is 20% chance of this being a label
                highlight = True #possibly an answer
        
            all_digits_prob.append(prob)
            
            if(plot):
            
                N, bins, patches = axs[j].hist(histo, bins=8, color = "lightgray", lw=0,  weights=np.ones(len(histo)) / len(histo), density=False)
                # N, bins, patches = axs[j].hist(histo, bins=(np.array([0, 1.,2.,3.,4.,5.,6.,7.,8.,9.,10.])*1/10.).tolist()    , color = "lightgray", lw=0,  weights=np.ones(len(histo)) / len(histo), density=False)
                # N, bins, patches = axs[j].hist(histo_exp, bins=10, color = None,   rwidth=100,align='mid', #lw=0,
                #                                weights=np.ones(len(histo)) / len(histo), density=False)
                # N, bins, patches = axs[j].hist(histo_exp, bins=2, color = "lightgray", histtype = 'stepfilled', lw=0,  weights=np.ones(len(histo)) / len(histo), density=True)
                axs[j].set_title(str(j)+" ("+str(round(prob,2))+")") 
        
            if(highlight):
            
                highted_something = True
                
                if(plot):

                    # We'll color code by height, but you could use any scalar
                    fracs = N / N.max()

                    # we need to normalize the data to 0..1 for the full range of the colormap
                    norm = colors.Normalize(fracs.min(), fracs.max())

                    # Now, we'll loop through our objects and set the color of each accordingly
                    for thisfrac, thispatch in zip(fracs, patches):
                        color = plt.cm.viridis(norm(thisfrac))
                        thispatch.set_facecolor(color)

    
        if(plot):
            plt.show()
    
        predicted = np.argmax(all_digits_prob)
    
        if(highted_something):
            predicted_for_images+=1
            if(labels[i].item()==predicted):
                if(plot):
                    print("Correct")
                correct_predictions +=1.0
            else:
                if(plot):
                    print(f"Incorrectly predicted as:{predicted}")
        else:
            if(plot):
                print("Undecided.")
        
        if(plot):
            imshow(images[i].squeeze())
        
    
    if(plot):
        print("Summary")
        print("Total images: ",len(labels))
        print("Predicted for: ",predicted_for_images)
        print("Accuracy when predicted: ",correct_predictions/predicted_for_images)
        
    return len(labels), correct_predictions, predicted_for_images 


    

In [ ]:
preds= Predictive(model = bcnn, guide = guide, num_samples = 100, return_sites=("_RETURN", "obs"))

for img, labels in test_loader:
  # samples = preds(img)
  break;
n_images = 20
test_image=img[:n_images]
test_label = labels[:n_images].numpy()
_, _, _ = test_batch(preds=preds, images=test_image, labels=test_label, plot=True)

  # predicted = np.argmax(samples['_RETURN'].mean(axis=[0]), axis=1)
  # predictions_all.extend(predicted.tolist())
  # correct_preds += (predicted == labels).sum()
  # if(num_batches_to_run_for == batch_num):
  #   break;
  # batch_num +=1




# Prediction when network can decide not to predict

# print('Prediction when network can refuse')
# correct = 0
# total = 0
# total_predicted_for = 0

# for img, labels in test_loader:
#     # images, labels = data
#     total_minibatch, correct_minibatch, predictions_minibatch = test_batch(preds, images, labels, plot=False)
#     total += total_minibatch
#     correct += correct_minibatch
#     total_predicted_for += predictions_minibatch

# print("Total images: ", total)
# print("Skipped: ", total-total_predicted_for)
# print("Accuracy when made predictions: %d %%" % (100 * correct / total_predicted_for))


# Random Noise as inputs


In [ ]:
preds= Predictive(model = bcnn, guide = guide, num_samples = 100, return_sites=("_RETURN", "obs"))
images_random = torch.rand(10,1,28,28)
labels_random = torch.randint(0,10, (10,))
test_batch(preds=preds, images=images_random, labels=labels_random, plot=True)

# logits = torch.tensor([-10.,-2.,-3.,-4.])
# smm = nn.Softmax(dim=-1)
# smm(logits).sum()


#Appendix/Trash

In [ ]:
# lets see if the parameters of the guide change.
# guide.requires_grad_(False)

for name, value in pyro.get_param_store().items():
    print(name)
    print(name, pyro.param(name))
    print(pyro.param(name).shape)
sum(p.numel() for p in guide.parameters())
8466*2

pyro.param("AutoDiagonalNormal.scale").sum()


# AutoDiagonalNormal.loc Parameter containing:
# tensor([ 1.1677, -0.0818,  0.0145,  ...,  0.0103, -0.0278, -0.1930],
#        requires_grad=True)
# torch.Size([8466])
# AutoDiagonalNormal.scale tensor([0.1279, 0.1280, 0.1281,  ..., 0.1281, 0.1281, 0.1280],
#        grad_fn=<AddBackward0>)
# torch.Size([8466])

In [ ]:
print('hi')